In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=cdad67d712ed8958ab9164daea440f4b799a05e5eba54e796176c9371543e2ca
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=2608ba06c738d61ed74d840eceecc2b77bb8dbddaf6a3b82e56dc4d5f973641b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
import pickle


LOG_DIR = f"{int(time.time())}"

tensorboard = TensorBoard(log_dir=LOG_DIR)

def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

# Hight and width of the images
IMAGE_SIZE = 32
# 3 channels, Red, Green and Blue
CHANNELS = 3
# Number of epochs
NUM_EPOCH = 10


# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = normalize(X_train)
X_test = normalize(X_test)
X_train = X_train.reshape(-1,32, 32, 3)  # reshaping for convnet


y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()

    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=X_train.shape[1:]))

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    for i in range(hp.Int('n_connections', 1, 4)):
        model.add(Dense(hp.Choice(f'n_nodes',
                                  values=[128, 256, 512, 1024])))
        model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model
    
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory='Parcial_DeepLearning',
    project_name='Prueba_2')

tuner.search_space_summary()

tuner.search(x=X_train,
             y=y_train,
             epochs=100,
             batch_size=32,
             validation_data=(X_test, y_test))

print(tuner.get_best_models()[0].summary())
print(tuner.get_best_hyperparameters()[0].values)

model = tuner.get_best_models(num_models=1)[0]
print (model.summary())
# Evaluate the best model.
loss, accuracy = model.evaluate(X_test, y_test)
print('loss:', loss)
print('accuracy:', accuracy)
model.save('prueba1.h5')


INFO:tensorflow:Reloading Oracle from existing project Parcial_DeepLearning/Prueba_2/oracle.json
INFO:tensorflow:Reloading Tuner from Parcial_DeepLearning/Prueba_2/tuner0.json


INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 192)       5376      
_________________________________________________________________
activation (Activation)      (None, 30, 30, 192)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 192)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 224)       387296    
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 224)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 192)       387264    
_________________________________________________________________
activation_2 (Acti

In [24]:
model.fit(X_train,y_train,batch_size=64,epochs= 100, validation_data=(X_test,y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('Fitmodel.h5')

Epoch 1/100
782/782 [==============================] - 10s 12ms/step - loss: 0.0485 - accuracy: 0.9920 - val_loss: 7.1393 - val_accuracy: 0.7156
Epoch 2/100
782/782 [==============================] - 10s 12ms/step - loss: 0.0689 - accuracy: 0.9900 - val_loss: 7.4231 - val_accuracy: 0.7097
Epoch 3/100
782/782 [==============================] - 9s 12ms/step - loss: 0.0562 - accuracy: 0.9905 - val_loss: 7.6098 - val_accuracy: 0.7114
Epoch 4/100
782/782 [==============================] - 10s 12ms/step - loss: 0.0557 - accuracy: 0.9913 - val_loss: 7.5002 - val_accuracy: 0.7101
Epoch 5/100
782/782 [==============================] - 9s 12ms/step - loss: 0.0481 - accuracy: 0.9924 - val_loss: 7.9152 - val_accuracy: 0.7125
Epoch 6/100
782/782 [==============================] - 9s 12ms/step - loss: 0.0471 - accuracy: 0.9931 - val_loss: 8.1222 - val_accuracy: 0.7081
Epoch 7/100
782/782 [==============================] - 9s 12ms/step - loss: 0.0515 - accuracy: 0.9921 - val_loss: 7.5510 - val_accura

NameError: ignored